# Belastingen uit RWS

#  importeer de benodigde functies

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from toolbox_continu_inzicht import Config

## 1. Voorbeeld met CSV
Lees de configuratie in:

Dit leest een configuratie bestand `belasting_rws_config.yaml` in. 
```yaml
GlobalVariables:
    rootdir: "tests/src/loads/data_sets" 
    moments: [-24,0,24,48]  

    LoadsWaterwebservicesRWS:
        parameters:  ['WATHTE']

DataAdapter: 
    default_options:
        csv:
            sep: ","
    BelastingLocaties: 
        type: csv
        path: "test_loads_rws_8420.csv"
    Waterstanden: 
        type: csv
        path: "hidden_waterstanden.csv"
...

```

In [ ]:
test_data_sets_path = Path.cwd() / "data_sets"
c = Config(config_path=test_data_sets_path / "loads_rws_config.yaml")
c.lees_config()

Zet de data adapter klaar en geef deze de configuratie mee 

In [ ]:
from toolbox_continu_inzicht import DataAdapter

data = DataAdapter(config=c)

Run de module met CSV

In [ ]:
from toolbox_continu_inzicht.loads import LoadsWaterwebservicesRWS

RWS_webservice = LoadsWaterwebservicesRWS(data_adapter=data)

In [ ]:
RWS_webservice.df_in

In [ ]:
lst_data = RWS_webservice.run(input="BelastingLocaties", output="Waterstanden")

In [ ]:
RWS_webservice.df_in

In [ ]:
df_out = RWS_webservice.df_out.copy()

In [ ]:
from toolbox_continu_inzicht.loads.loads_rws_webservice.get_rws_webservices_locations import (
    get_rws_webservices_locations,
)

df_loc = get_rws_webservices_locations()

In [ ]:
url = "https://ddapi20-waterwebservices.rijkswaterstaat.nl/ONLINEWAARNEMINGENSERVICES/OphalenWaarnemingen"

In [ ]:
d = {
    "Locatie": {"Code": "rosmalen.hinthamsluis.boven"},
    "AquoPlusWaarnemingMetadata": {
        "AquoMetadata": {
            "Compartiment": {"Code": "OW"},
            "Grootheid": {"Code": "WATHTE"},
            "ProcesType": "meting",
        }
    },
    "Periode": {
        "Begindatumtijd": "2025-12-21T16:00:00.000+00:00",
        "Einddatumtijd": "2025-12-24T16:00:00.000+00:00",
    },
}

In [ ]:
import httpx

with httpx.Client() as client:
    response = client.post(url=url, json=d)

In [ ]:
response

In [ ]:
df_loc[df_loc["Code"] == "dekrim"]

In [ ]:
import folium

from IPython.display import display, HTML


def folium_show(m):
    data = m.get_root().render()
    data_fixed_height = data.replace("width: 100%;height: 100%", "width: 100%").replace(
        "height: 100.0%;", "height: 609px;", 1
    )
    display(HTML(data_fixed_height))


df_location = df_loc
# Create a map centered at a specific location
m = folium.Map(
    location=[df_location.Lat.mean(), df_location.Lon.mean()],
    zoom_start=10,
)

# Add markers to the map
for _, row in df_location.iterrows():
    popup_text = f"Code_int: {row.name}<br>Code.name: {row['Code']}"
    folium.Marker(location=[row.Lat, row.Lon], popup=popup_text).add_to(m)

m

In [ ]:
# split de dataframe in tweeën & process
values_nan = df_out[df_out["value"] == -999].index
for val in values_nan:
    df_out.loc[val, "value"] = np.nan
df_out.sort_index()
df_plot = df_out.set_index(df_out["date_time"])
df_plot = df_plot[df_plot["measurement_location_code"] == 8420]
measurement_index = df_plot["value_type"] == "verwachting"
df_plot_measurements = df_plot[measurement_index][["value"]]
df_plot_forecast = df_plot[~measurement_index][["value"]]

In [ ]:
fig, ax = plt.subplots()
df_plot_measurements.plot(color="C0", ax=ax)
df_plot_forecast.plot(color="C1", ax=ax)
ax.legend(["Prediction", "Measurement"])